In [ ]:
%mkdir /root/.kaggle
%mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c playground-series-s4e10

 69% 1.00M/1.45M [00:00<00:00, 2.10MB/s]
100% 1.45M/1.45M [00:00<00:00, 2.74MB/s]


In [ ]:
!unzip playground-series-s4e10.zip

Archive:  playground-series-s4e10.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingClassifier
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from optuna.samplers import TPESampler

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
!kaggle datasets download chilledwanker/loan-approval-prediction

Dataset URL: https://www.kaggle.com/datasets/chilledwanker/loan-approval-prediction
License(s): apache-2.0
100% 368k/368k [00:00<00:00, 972kB/s]
100% 368k/368k [00:00<00:00, 971kB/s]


In [ ]:
!unzip loan-approval-prediction.zip

Archive:  loan-approval-prediction.zip
  inflating: credit_risk_dataset.csv  


In [ ]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
original_data = pd.read_csv("credit_risk_dataset.csv")

In [ ]:
print(data.shape)
print(test_data.shape)
print(original_data.shape)

(58645, 13)
(39098, 12)
(32581, 12)


In [ ]:
original_data.isna().any()

,0
person_age,False
person_income,False
person_home_ownership,False
person_emp_length,True
loan_intent,False
loan_grade,False
loan_amnt,False
loan_int_rate,True
loan_status,False
loan_percent_income,False


In [ ]:
data = pd.concat([data.drop("id",axis=1),original_data],axis=0)

In [ ]:
data['loan_status'].value_counts()

,count
loan_status,
0,75768
1,15458


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [ ]:
ids = test_data['id']
test_data.drop('id',axis=1,inplace=True)

In [ ]:
from sklearn.utils import resample

loan = data.loc[data["loan_status"]==1]
non_loan = data.loc[data["loan_status"]==0]

non_loan_downsampeld = resample(non_loan,n_samples=loan.shape[0]+1000,random_state=42)

data = pd.concat([loan,non_loan_downsampeld])

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
for col in data.select_dtypes(include='object').columns.tolist():
  data[col] = data[col].astype("category")

In [ ]:
for col in test_data.select_dtypes(include='object').columns.tolist():
  test_data[col] = test_data[col].astype("category")

#Analaysis

In [ ]:
cat_X_train,cat_X_test,cat_y_train,cat_y_test = train_test_split(data.drop('loan_status',axis=1),data['loan_status'],test_size=0.1,random_state=42)

In [ ]:
def model_test():
  cat_features = data.select_dtypes(include='category').columns.tolist()
  temp_model = CatBoostClassifier(cat_features=cat_features,silent=True)
  temp_model.fit(cat_X_train,cat_y_train)

  prediction = temp_model.predict(cat_X_test)
  roc = roc_auc_score(cat_y_test,prediction)
  print(f"roc : {roc}")
  return temp_model
temp_model = model_test()

roc : 0.9006300975531744


In [ ]:
def predict_and_save(model,test_data):
  sub = pd.DataFrame()
  sub['id'] = ids
  sub['loan_status'] = model.predict(test_data)
  sub.to_csv('submission.csv',index=False)
  return sub
#sub = predict_and_save(temp_model)

In [ ]:
def objective_cat(trial):

  catboost_params = {
      "iterations": trial.suggest_int("iterations", 100, 5000, step=100),
      "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
      "depth": trial.suggest_int("depth", 1, 15),
      "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
      "rsm": trial.suggest_float("rsm", 0.05, 1.0),
      "subsample": trial.suggest_float("subsample", 0.05, 1.0),
      "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
      "loss_function": "Logloss",  # For binary classification
      "verbose": 0
  }
  cat_features = data.select_dtypes(include='category').columns.tolist()
  model = CatBoostClassifier(cat_features=cat_features,silent=True)
  scores = cross_val_score(model,data.drop('loan_status',axis=1),data['loan_status'],cv=5,scoring="roc_auc")
  return scores.mean()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_cat, n_trials=10)

[I 2024-10-28 22:34:27,251] A new study created in memory with name: no-name-40ff5cdc-deb6-4310-a578-a77affd7c0a8
[I 2024-10-28 22:36:20,233] Trial 0 finished with value: 0.9554658170290435 and parameters: {'iterations': 1200, 'learning_rate': 0.02893698238923492, 'depth': 8, 'min_data_in_leaf': 21, 'rsm': 0.9328891287180453, 'subsample': 0.4138256931518198, 'l2_leaf_reg': 1.0116664002968488e-07}. Best is trial 0 with value: 0.9554658170290435.
[I 2024-10-28 22:38:11,654] Trial 1 finished with value: 0.9554658170290435 and parameters: {'iterations': 900, 'learning_rate': 0.019342030095037984, 'depth': 14, 'min_data_in_leaf': 60, 'rsm': 0.1619389518233245, 'subsample': 0.9866489774161024, 'l2_leaf_reg': 7.34751048065774e-05}. Best is trial 0 with value: 0.9554658170290435.
[I 2024-10-28 22:40:02,082] Trial 2 finished with value: 0.9554658170290435 and parameters: {'iterations': 4300, 'learning_rate': 0.0034367409859419273, 'depth': 2, 'min_data_in_leaf': 99, 'rsm': 0.826055814129686, 's

In [ ]:
cat_features = data.select_dtypes(include='category').columns.tolist()
model = CatBoostClassifier(cat_features=cat_features,**study.best_params,silent=True,loss_function= "Logloss")
model.fit(data.drop('loan_status',axis=1),data['loan_status'])

In [ ]:
sub = predict_and_save(model)

In [ ]:
def encode_labels(data):
  cols = data.select_dtypes(include='category').columns.tolist()
  for col in cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    print(f"{le.classes_} : {le.transform(le.classes_)}")
  return data

data_xgb = encode_labels(data.copy())


['MORTGAGE' 'OTHER' 'OWN' 'RENT'] : [0 1 2 3]
['DEBTCONSOLIDATION' 'EDUCATION' 'HOMEIMPROVEMENT' 'MEDICAL' 'PERSONAL'
 'VENTURE'] : [0 1 2 3 4 5]
['A' 'B' 'C' 'D' 'E' 'F' 'G'] : [0 1 2 3 4 5 6]
['N' 'Y'] : [0 1]


In [ ]:
test_data_xgb = encode_labels(test_data.copy())

['MORTGAGE' 'OTHER' 'OWN' 'RENT'] : [0 1 2 3]
['DEBTCONSOLIDATION' 'EDUCATION' 'HOMEIMPROVEMENT' 'MEDICAL' 'PERSONAL'
 'VENTURE'] : [0 1 2 3 4 5]
['A' 'B' 'C' 'D' 'E' 'F' 'G'] : [0 1 2 3 4 5 6]
['N' 'Y'] : [0 1]


##XGB MODEL

In [ ]:
X = data.drop('loan_status',axis=1)
y = data['loan_status']

In [ ]:
X

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3
...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10
58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8
58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2
58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3


In [ ]:
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': 1000,
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'random_state': 42,
        'n_jobs': -1,
        'tree_method': 'gpu_hist',
        'verbosity': 0,  # Suppress warnings and messages from XGBoost
        'enable_categorical':True,
    }

    # Fitting LGBM model with parameters from the trials
    model = XGBClassifier(**params)

    # Stratified sampling
    cv = StratifiedKFold(5, shuffle=True, random_state=42)
    cv_splits = cv.split(X, y)

    # Creating empty scores list to hold AUC scores from each trialed model
    scores = []
    for train_idx, val_idx in cv_splits:
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(X_train_fold, y_train_fold)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1]
        score = roc_auc_score(y_val_fold, y_pred_proba)
        scores.append(score)

    # Printing and returning mean AUC scores
    mean_score = np.mean(scores)
    print(f"Mean ROC AUC Score = {mean_score:.5f}")
    return mean_score

In [ ]:
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=42), direction="maximize")
study.optimize(objective, n_trials=100)
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)


/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:30: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-10-29 15:03:39,743] A new study created in memory with name: no-name-28c82b15-d076-4c5b-a61a-676175c38b73
[I 2024-10-29 15:04:26,046] Trial 0 finished with value: 0.9535898363700406 and parameters: {'learning_rate': 0.03574712922600244, 'max_depth': 12, 'subsample': 0.892797576724562, 'colsample_bytree': 0.8394633936788146, 'lambda': 0.004207988669606638, 'alpha': 0.004207053950287938}. Best is trial 0 with value: 0.9535898363700406.


Mean ROC AUC Score = 0.95359


[I 2024-10-29 15:04:59,201] Trial 1 finished with value: 0.952831406705306 and parameters: {'learning_rate': 0.012184186502221764, 'max_depth': 12, 'subsample': 0.8404460046972835, 'colsample_bytree': 0.8832290311184181, 'lambda': 0.0012087541473056963, 'alpha': 7.579479953348009}. Best is trial 0 with value: 0.9535898363700406.


Mean ROC AUC Score = 0.95283


[I 2024-10-29 15:05:19,313] Trial 2 finished with value: 0.9490397707259277 and parameters: {'learning_rate': 0.16967533607196555, 'max_depth': 7, 'subsample': 0.6727299868828402, 'colsample_bytree': 0.6733618039413735, 'lambda': 0.016480446427978974, 'alpha': 0.12561043700013558}. Best is trial 0 with value: 0.9535898363700406.


...

Mean ROC AUC Score = 0.95565


[I 2024-10-29 15:43:34,744] Trial 98 finished with value: 0.9569062089724294 and parameters: {'learning_rate': 0.017911266082033486, 'max_depth': 6, 'subsample': 0.9789047511069877, 'colsample_bytree': 0.7167539860298056, 'lambda': 0.0011033422336761485, 'alpha': 0.4117027121415347}. Best is trial 78 with value: 0.9572391687213807.


Mean ROC AUC Score = 0.95691


[I 2024-10-29 15:43:55,272] Trial 99 finished with value: 0.9570622178103687 and parameters: {'learning_rate': 0.01828211105198083, 'max_depth': 7, 'subsample': 0.9985718129347925, 'colsample_bytree': 0.6442523822565819, 'lambda': 0.0013106550888030265, 'alpha': 0.0013140120861356538}. Best is trial 78 with value: 0.9572391687213807.


Mean ROC AUC Score = 0.95706
Best value: 0.9572391687213807
Best trial: {'learning_rate': 0.020072221025811497, 'max_depth': 7, 'subsample': 0.98736802503195, 'colsample_bytree': 0.6410084471560525, 'lambda': 0.0021021455992778246, 'alpha': 1.294966468926271}


In [ ]:
params = {'learning_rate': 0.020072221025811497, 'max_depth': 7, 'subsample': 0.98736802503195, 'colsample_bytree': 0.6410084471560525, 'lambda': 0.0021021455992778246, 'alpha': 1.294966468926271}


In [ ]:
model = XGBClassifier(**params,random_state= 42,n_jobs= -1,tree_method= "gpu_hist",verbosity= 0,enable_categorical=True,objective= 'binary:logistic',eval_metric='auc',n_estimators=3000)
model.fit(X,y)

XGBClassifier(alpha=1.294966468926271, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6410084471560525, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='auc', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.0021021455992778246,
              learning_rate=0.020072221025811497, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=-1, ...)

In [ ]:
sub = pd.DataFrame()
sub['id'] = ids
sub['loan_status'] = model.predict(test_data)
sub.to_csv('submission.csv',index=False)

In [ ]:
!kaggle competitions submit -c playground-series-s4e10 -f submission.csv -m "submission"

100% 305k/305k [00:00<00:00, 422kB/s]
Successfully submitted to Loan Approval Prediction